In [94]:
from SmartApi.smartConnect import SmartConnect
import pyotp
import pandas as pd
import requests
import pandas as pd
from datetime import datetime, timedelta
from datetime import datetime, timedelta
import pandas_ta as ta


api_key = 'C46AImmE'
clientId = 'M507690'
pwd = '4663'
smartApi = SmartConnect(api_key)
token = "4ACNNSIZL6EV6UQ2ACSI23H5VQ"
totp=pyotp.TOTP(token).now()
correlation_id = "abc123"

# login api call

data = smartApi.generateSession(clientId, pwd, totp)
# print(data)
authToken = data['data']['jwtToken']
refreshToken = data['data']['refreshToken']

# fetch the feedtoken
feedToken = smartApi.getfeedToken()

# fetch User Profile
res = smartApi.getProfile(refreshToken)
smartApi.generateToken(refreshToken)
res=res['data']['exchanges']

#fetch User Profile
userProfile= smartApi.getProfile(refreshToken)
User= pd.DataFrame(userProfile)
User = User['data']['name']
User
print(f"Welcome {User} to the World of SMART API BY ANGEL.....!!")
print(f"Login Sucessfull....!!")


[I 251222 01:19:59 smartConnect:124] in pool


Welcome MANA  BHATTACHARYYA to the World of SMART API BY ANGEL.....!!
Login Sucessfull....!!


In [95]:
# ---- NIFTY 50 Token (Angel One Fixed Token) ----
NIFTY_TOKEN = "26000"      # NIFTY 50 Index
EXCHANGE = "NSE"

# ---- Fetch LTP ----
ltp_data = smartApi.ltpData(
    exchange=EXCHANGE,
    tradingsymbol="NIFTY",
    symboltoken=NIFTY_TOKEN
)

print("📈 NIFTY 50 LTP:", ltp_data['data']['ltp'])

ATM = round(ltp_data['data']['ltp'] / 50) * 50
print("📈 NIFTY 50 SPOT Price:", ATM)

ltp =  ltp_data['data']['ltp']
prev_close = ltp_data['data']['close']

change = ltp - prev_close
pct_change = (change / prev_close) * 100

print(f"NIFTY Change: {change:.2f}")
print(f"NIFTY % Change: {pct_change:.2f}%")


📈 NIFTY 50 LTP: 25966.4
📈 NIFTY 50 SPOT Price: 25950
NIFTY Change: 150.85
NIFTY % Change: 0.58%


In [96]:
import pandas as pd
import pandas_ta as ta

def add_indicators(df):
    df["VWAP"] = (df["Close"] * df["Volume"]).cumsum() / df["Volume"].cumsum()
    df["EMA14"] = ta.ema(df["Close"], length=20)
    df["RSI"] = ta.rsi(df["Close"], length=14)

    macd = ta.macd(df["Close"])
    df["MACD"] = macd["MACD_12_26_9"]
    df["MACD_SIGNAL"] = macd["MACDs_12_26_9"]

    return df

In [97]:
url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
instruments = pd.DataFrame(requests.get(url).json())
instruments.to_csv('Token_df.csv')

expiry = instruments[(instruments['name']=='NIFTY') &  (instruments['instrumenttype']=='OPTIDX') &  (instruments['exch_seg']=='NFO')]["expiry"].unique()

expiry

today = datetime.today()

dates = pd.to_datetime(expiry, format="%d%b%Y")
next_expiry = dates[dates > today].min()
print("Next Expiry:", next_expiry.strftime("%d%b%y").upper())

Next Expiry: 23DEC25


In [98]:
expiry_next = next_expiry.strftime("%d%b%y").upper()

def get_option_symbol(expiry_next, strike, opt_type):
    return f"NIFTY{expiry_next}{strike}{opt_type}"

ce_symbol = get_option_symbol(expiry_next, ATM+100, "CE")
pe_symbol = get_option_symbol(expiry_next, ATM-100, "PE")

print("CE Symbol:", ce_symbol)
print("PE Symbol:", pe_symbol)

CE Symbol: NIFTY23DEC2526050CE
PE Symbol: NIFTY23DEC2525850PE


In [99]:
def get_option_token(symbol):
    row = instruments[instruments["symbol"] == symbol]
    if row.empty:
        raise Exception(f"Token not found for {symbol}")
    return row.iloc[0]["token"]

ce_token = get_option_token(ce_symbol)
pe_token = get_option_token(pe_symbol)

print("CE Token:", ce_token)
print("PE Token:", pe_token)

CE Token: 57009
PE Token: 56995


In [100]:
def fetch_5min_candles(token):
    to_date = datetime.now()
    from_date = to_date - timedelta(days=10)

    response = smartApi.getCandleData({
        "exchange": "NFO",
        "symboltoken": token,
        "interval": "THREE_MINUTE",
        "fromdate": from_date.strftime("%Y-%m-%d %H:%M"),
        "todate": to_date.strftime("%Y-%m-%d %H:%M")
    })

    if not response or "data" not in response or response["data"] is None:
        raise Exception("No candle data")

    df = pd.DataFrame(
        response["data"],
        columns=["Datetime", "Open", "High", "Low", "Close", "Volume"]
    )

    # 🔑 CRITICAL FIX
    df["Datetime"] = pd.to_datetime(df["Datetime"])
    df.set_index("Datetime", inplace=True)
    df.sort_index(inplace=True)

    return df



def apply_indicators(df):
    df = df.copy()

    df["EMA14"] = ta.ema(df["Close"], length=5)
    df["EMA20"] = ta.ema(df["Close"], length=10)
    df["RSI"] = ta.rsi(df["Close"], length=14)

    df["VWAP"] = ta.vwap(
        high=df["High"],
        low=df["Low"],
        close=df["Close"],
        volume=df["Volume"]
    )

    df.dropna(inplace=True)
    return df

ce_df = fetch_5min_candles(ce_token)
pe_df = fetch_5min_candles(pe_token)

ce_df = apply_indicators(ce_df)
pe_df = apply_indicators(pe_df)

In [101]:
NIFTY1 = ce_df.copy()
import numpy as np
import time
import numpy as np
from datetime import datetime, time
from datetime import time as dt_time

# ---------------- INITIALIZE COLUMNS ----------------
NIFTY1 = NIFTY1.copy()
NIFTY1["signal"] = 0
NIFTY1["position"] = 0
NIFTY1["entry_price"] = np.nan
NIFTY1["exit_price"] = np.nan
NIFTY1["exit_reason"] = ""
NIFTY1["pnl"] = 0.0

# ---------------- SELL SIGNAL (EMA < VWAP CROSSDOWN) ----------------
sell_condition = (
    (NIFTY1["EMA14"] < NIFTY1["VWAP"]) & (NIFTY1["RSI"] < 55) &
    (NIFTY1["EMA14"].shift(1) >= NIFTY1["VWAP"].shift(1))
)

NIFTY1.loc[sell_condition, "signal"] = -1

# ---------------- TRADE MANAGEMENT ----------------
ENTRY_START = dt_time(9, 45)
ENTRY_END   = dt_time(13, 0)
EXIT_TIME   = dt_time(15, 0)

in_trade = False
sell_price = None

for i in range(len(NIFTY1)):

    idx   = NIFTY1.index[i]
    tm    = idx.time()
    price = NIFTY1["EMA14"].iloc[i]
    rsi   = NIFTY1["RSI"].iloc[i]

    # -------- ENTRY (SELL) --------
    if (
        not in_trade and
        ENTRY_START <= tm < ENTRY_END and
        price < NIFTY1["VWAP"].iloc[i] and
        rsi < 55 and
        rsi > 30

    ):
        in_trade = True
        sell_price = price

        NIFTY1.at[idx, "position"] = -1
        NIFTY1.at[idx, "entry_price"] = sell_price

    # -------- EXIT --------
    elif in_trade:

        # ⏰ 3:00 PM AUTO EXIT
        if tm >= EXIT_TIME:
            in_trade = False
            NIFTY1.at[idx, "exit_price"] = price
            NIFTY1.at[idx, "exit_reason"] = "3:00 PM Square-off"
            NIFTY1.at[idx, "pnl"] = sell_price - price

        # ✅ PROFIT BOOKING
        elif rsi < 25:
            in_trade = False
            NIFTY1.at[idx, "exit_price"] = price
            NIFTY1.at[idx, "exit_reason"] = "RSI < 25 (Profit)"
            NIFTY1.at[idx, "pnl"] = sell_price - price

        # ❌ STOP LOSS
        elif price > 1.15 * sell_price:
            in_trade = False
            NIFTY1.at[idx, "exit_price"] = price
            NIFTY1.at[idx, "exit_reason"] = "SL Hit (15%)"
            NIFTY1.at[idx, "pnl"] = sell_price - price

NIFTY1.to_csv('Trades.csv')


In [102]:
NIFTY1["pnl"] = 0.0

for i in range(len(NIFTY1)):
    if (
        pd.notna(NIFTY1["exit_price"].iloc[i]) and
        pd.notna(NIFTY1["entry_price"].iloc[i])
    ):
        entry = NIFTY1["entry_price"].iloc[i]
        exitp = NIFTY1["exit_price"].iloc[i]

        # OPTION SELL → profit when price falls
        pnl = entry - exitp

        NIFTY1.at[NIFTY1.index[i], "pnl"] = pnl


In [103]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig1 = make_subplots(
rows=2,
cols=1,
shared_xaxes=True,
vertical_spacing=0.08,
row_heights=[0.7, 0.3],
#subplot_titles=("Price + VWAP + EMA + Signals", "RSI")
)

# -------- Candlestick --------
fig1.add_trace(
go.Candlestick(
    x=NIFTY1.index,
    open=NIFTY1["Open"],
    high=NIFTY1["High"],
    low=NIFTY1["Low"],
    close=NIFTY1["Close"],
    name="Price"
),
row=1, col=1
)

# -------- EMA14 --------
fig1.add_trace(
go.Scatter(
    x=NIFTY1.index,
    y=NIFTY1["EMA14"],
    mode="lines",
    line=dict(color="cyan", width=2),
    name="EMA14"
),
row=1, col=1
)

# -------- VWAP --------
fig1.add_trace(
go.Scatter(
    x=NIFTY1.index,
    y=NIFTY1["VWAP"],
    mode="lines",
    line=dict(color="orange", dash="dot", width=2),
    name="VWAP"
),
row=1, col=1
)

# -------- BUY Signals --------
fig1.add_trace(
go.Scatter(
    x=NIFTY1[NIFTY1["signal"] == 1].index,
    y=NIFTY1[NIFTY1["signal"] == 1]["Close"],
    mode="markers",
    marker=dict(color="green", size=16, symbol="triangle-up"),
    name="BUY"
),
row=1, col=1
)

# -------- SELL Signals --------
fig1.add_trace(
go.Scatter(
    x=NIFTY1[NIFTY1["signal"] == -1].index,
    y=NIFTY1[NIFTY1["signal"] == -1]["Close"],
    mode="markers",
    marker=dict(color="red", size=16, symbol="triangle-down"),
    name="SELL"
),
row=1, col=1
)

# -------- RSI --------
fig1.add_trace(
go.Scatter(
    x=NIFTY1.index,
    y=NIFTY1["RSI"],
    mode="lines",
    line=dict(color="orange", width=2),
    name="RSI"
),
row=2, col=1
)

# Profit exits
fig1.add_trace(
    go.Scatter(
        x=NIFTY1[NIFTY1["exit_reason"].str.contains("Profit", na=False)].index,
        y=NIFTY1[NIFTY1["exit_reason"].str.contains("Profit", na=False)]["Close"],
        mode="markers",
        marker=dict(color="lime", size=14, symbol="circle"),
        name="BOOK PROFIT"
    ),
    row=1, col=1
)

# SL exits
fig1.add_trace(
    go.Scatter(
        x=NIFTY1[NIFTY1["exit_reason"].str.contains("SL", na=False)].index,
        y=NIFTY1[NIFTY1["exit_reason"].str.contains("SL", na=False)]["Close"],
        mode="markers",
        marker=dict(color="red", size=14, symbol="x"),
        name="STOP LOSS"
    ),
    row=1, col=1
)

fig1.add_hline(y=70, row=2, col=1, line_dash="dash", line_color="red")
fig1.add_hline(y=30, row=2, col=1, line_dash="dash", line_color="green")

# -------- Layout --------
fig1.update_layout(
height=750,
template="plotly_dark",
xaxis_rangeslider_visible=False,
hovermode="x unified",
legend=dict(orientation="h", y=1.08)
)
fig1.show()



In [104]:
NIFTY1 = pd.read_csv('Trades.csv')
NIFTY2 = NIFTY1[(NIFTY1['entry_price'] > 0) | (NIFTY1['exit_price'] > 0)]
NIFTY2

,Datetime,Open,High,Low,Close,Volume,EMA14,EMA20,RSI,VWAP,signal,position,entry_price,exit_price,exit_reason,pnl
13,2025-12-12 10:21:00+05:30,173.00,174.30,153.25,153.50,384,163.676725,166.888093,50.859083,173.890299,0,-1,163.676725,NaN,NaN,0.000000
106,2025-12-12 15:00:00+05:30,177.75,184.85,177.00,179.30,410,176.822992,176.461124,53.860648,171.108307,0,0,NaN,176.822992,3:00 PM Square-off,-13.146268
127,2025-12-15 09:48:00+05:30,118.70,121.05,116.95,120.25,465,118.787640,123.989408,30.734545,122.682722,0,-1,118.787640,NaN,NaN,0.000000
169,2025-12-15 11:54:00+05:30,146.00,148.40,143.00,147.80,578,139.116022,134.454945,65.042040,127.005474,0,0,NaN,139.116022,SL Hit (15%),-20.328382
261,2025-12-16 10:15:00+05:30,90.30,92.70,88.50,92.00,1110,91.645689,94.254821,30.509911,99.502190,0,-1,91.645689,NaN,NaN,0.000000
356,2025-12-16 15:00:00+05:30,76.85,77.55,72.40,72.40,2319,74.765670,75.357223,35.021097,87.936957,0,0,NaN,74.765670,3:00 PM Square-off,16.880018
376,2025-12-17 09:45:00+05:30,65.00,68.50,63.05,63.05,13891,69.854003,71.708465,36.071775,73.812672,0,-1,69.854003,NaN,NaN,0.000000
481,2025-12-17 15:00:00+05:30,43.00,45.50,42.70,42.85,11117,42.193842,41.468481,58.069076,54.789809,0,0,NaN,42.193842,3:00 PM Square-off,27.660161
506,2025-12-18 10:00:00+05:30,25.15,25.85,24.55,25.50,3355,25.361388,26.859132,31.193493,30.159764,0,-1,25.361388,NaN,NaN,0.000000
511,2025-12-18 10:15:00+05:30,31.20,37.30,29.70,35.20,12686,31.008907,29.502975,58.812092,30.315086,0,0,NaN,31.008907,SL Hit (15%),-5.647519


In [105]:
Lots = 1*75
Trade = NIFTY1[(NIFTY1['pnl'] != 0)]
Total_Trades = len(Trade)
Total_Profit_Trades= len(Trade[Trade['pnl'] > 0])
Win_Rate = (Total_Profit_Trades/Total_Trades)*100
total_Income = (NIFTY1['pnl'].sum())*Lots

print(f"Total No of Trades : {Total_Trades}")
print(f"Total No of Profit Trade : {Total_Profit_Trades}")
print(f"Win Rate : {round(Win_Rate,2)}%")
print(f"Total Income(Rs/-) : {round(total_Income,2)}")

Trade

Total No of Trades : 8
Total No of Profit Trade : 3
Win Rate : 37.5%
Total Income(Rs/-) : 1010.71


,Datetime,Open,High,Low,Close,Volume,EMA14,EMA20,RSI,VWAP,signal,position,entry_price,exit_price,exit_reason,pnl
106,2025-12-12 15:00:00+05:30,177.75,184.85,177.00,179.30,410,176.822992,176.461124,53.860648,171.108307,0,0,NaN,176.822992,3:00 PM Square-off,-13.146268
169,2025-12-15 11:54:00+05:30,146.00,148.40,143.00,147.80,578,139.116022,134.454945,65.042040,127.005474,0,0,NaN,139.116022,SL Hit (15%),-20.328382
356,2025-12-16 15:00:00+05:30,76.85,77.55,72.40,72.40,2319,74.765670,75.357223,35.021097,87.936957,0,0,NaN,74.765670,3:00 PM Square-off,16.880018
481,2025-12-17 15:00:00+05:30,43.00,45.50,42.70,42.85,11117,42.193842,41.468481,58.069076,54.789809,0,0,NaN,42.193842,3:00 PM Square-off,27.660161
511,2025-12-18 10:15:00+05:30,31.20,37.30,29.70,35.20,12686,31.008907,29.502975,58.812092,30.315086,0,0,NaN,31.008907,SL Hit (15%),-5.647519
665,2025-12-19 11:42:00+05:30,39.55,39.90,32.90,33.45,43534,38.457856,41.467589,22.566476,51.136449,0,0,NaN,38.457856,RSI < 25 (Profit),15.629234
686,2025-12-19 12:45:00+05:30,43.75,44.75,43.20,43.45,11189,43.158398,42.186948,53.782363,48.474947,0,0,NaN,43.158398,SL Hit (15%),-5.754906
731,2025-12-19 15:00:00+05:30,42.95,45.30,39.95,44.25,52221,44.188504,44.661958,48.856814,47.020683,0,0,NaN,44.188504,3:00 PM Square-off,-1.816239
